In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
from numpy import load
from google.colab import drive
import os
import math

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def build_siamese_model(inputShape, embeddingDim=48):
    # specify the inputs for the feature extractor network
    inputs = Input(inputShape)
    # define the first set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (3, 4), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(3, 4))(x)
    x = Dropout(0.3)(x)
    # second set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (3, 4), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=3)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    # second set of CONV => RELU => POOL => DROPOUT layers
    x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    # prepare the final outputs
    pooledOutput = GlobalAveragePooling2D()(x)


    outputs = Dense(embeddingDim)(pooledOutput)
    # build the model
    model = Model(inputs, outputs)
    # model.summary()
    # return the model to the calling function
    return model
  
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
        keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

img_shape = (217, 334, 1)

In [ ]:
loaded = np.load('dataset_path.npz')

In [ ]:
a = loaded['a']
b = loaded['b']
c =loaded['c']
d = loaded['d']

numpy.float16

In [ ]:
x_train = loaded['a']
x_test = loaded['b']
y_train =loaded['c']
y_test = loaded['d']

In [ ]:
x_train.shape

(20000, 2, 217, 334)

In [ ]:
def plot_loss(H):
  # construct a plot that plots and saves the training history
  plt.style.use("ggplot")
  plt.figure()
  plt.plot(H.history["loss"], label="train_loss")
  plt.plot(H.history["val_loss"], label="val_loss")
  # plt.plot(H.history["accuracy"], label="train_acc")
  # plt.plot(H.history["val_accuracy"], label="val_acc")
  plt.title("Training vs Validation Loss ")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss")
  plt.legend(loc="lower left")

def plot_accuracy(H):
  # construct a plot that plots and saves the training history
  plt.style.use("ggplot")
  plt.figure()
  # plt.plot(H.history["loss"], label="train_loss")
  # plt.plot(H.history["val_loss"], label="val_loss")
  plt.plot(H.history["accuracy"], label="train_acc")
  plt.plot(H.history["val_accuracy"], label="val_acc")
  plt.title("Training vs Validation accuracy ")
  plt.xlabel("Epoch #")
  plt.ylabel("Accuracy")
  plt.legend(loc="lower left")

In [ ]:
imgA = Input(shape=img_shape)
imgB = Input(shape=img_shape)

featureExtractor = build_siamese_model(img_shape)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)

model = Model(inputs=[imgA, imgB], outputs=outputs)

epochs = 180  
learning_rate = 0.0008

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate),
    metrics=["accuracy"])

# model.summary()


In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=32, 
    epochs=epochs)

In [ ]:
history = model.fit(
    a[:21000], b[:21000],
    validation_data=(c, d),
    batch_size=16, 
    epochs=epochs,)

plot_loss(history)
plot_accuracy(history)


In [ ]:
model.save('/content/gdrive/MyDrive/Music Recommendation/models/balanced/')
model.save_weights('/content/gdrive/MyDrive/Music Recommendation/weights/balanced/balanced')

In [ ]:
plot_loss(history)
plot_accuracy(history)

In [ ]:
keras.backend.clear_session()

In [ ]:
batch_x_train = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_x_train.npz")
batch_y_train = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_y_train.npz")
batch_x_test  = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_x_test.npz")
batch_y_test  = np.load("/content/gdrive/MyDrive/Music Recommendation/datasets/32k/batch_y_test.npz")

In [ ]:
class PairsGenerator(keras.utils.Sequence):
    def __init__ (self, x_dataset, y_dataset):
        self.x_dataset = x_dataset
        self.y_dataset = y_dataset
    
    def __len__(self):
        return len(self.x_dataset)
    
    def __getitem__ (self, idx):
        return [self.x_dataset['arr_'+str(idx)][:,0], self.x_dataset['arr_'+str(idx)][:,1]], self.y_dataset['arr_'+str(idx)]
        

In [ ]:
training_batch_generator = PairsGenerator(batch_x_train, batch_y_train)
testing_batch_generator = PairsGenerator(batch_x_test, batch_y_test)

In [ ]:
imgA = Input(shape=img_shape)
imgB = Input(shape=img_shape)


featureExtractor = build_siamese_model(img_shape)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)

model = Model(inputs=[imgA, imgB], outputs=outputs)

epochs = 180  
learning_rate = 0.0008

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate),
    metrics=["accuracy"])

# model.summary()


In [ ]:
checkpoint_filepath = '/content/gdrive/MyDrive/Music Recommendation/tmp_model/tmp_model2'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
model.fit(
    training_batch_generator,
    validation_data=testing_batch_generator,
    batch_size=32, 
    epochs=150,
    callbacks=[model_checkpoint_callback])

Epoch 1/150
839/839 [==============================] - 154s 184ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6932 - val_accuracy: 0.4951
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Music Recommendation/tmp_model/tmp_model2/assets
Epoch 2/150
230/839 [=======>......................] - ETA: 1:32 - loss: 0.6932 - accuracy: 0.4891

KeyboardInterrupt: ignored

In [ ]:
array = np.array([[1,2], [3,4]])
min_v = np.min(array)
max_v = np.max(array)
array = (array-min_v)/(max_v-min_v)

In [ ]:
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Music Recommendation/tmp_model/tmp_model')
checkpoint_filepath = '/content/gdrive/MyDrive/Music Recommendation/tmp_model/tmp_model'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
model.fit(
    training_batch_generator,
    validation_data=testing_batch_generator,
    batch_size=32, 
    epochs=30,
    callbacks=[model_checkpoint_callback])

In [ ]:
import gc

gc.collect()

131